<a href="https://colab.research.google.com/github/ovbystrova/hse_compling/blob/main/hw2/hw2_collocations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Data

In [1]:
%%capture
!pip install razdel

In [2]:
%%capture
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/2ch_corpus.txt.zip  -O dvach_corpus.txt.zip
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/lenta.txt.zip -O lenta.txt.zip

!unzip dvach_corpus.txt.zip
!unzip lenta.txt.zip

In [9]:
import gensim
import numpy as np
from nltk.tokenize import sent_tokenize
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from collections import Counter
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
fname_lenta = 'lenta.txt'
fname_2ch = '2ch_corpus.txt'

In [11]:
with open(fname_lenta, 'r', encoding='utf-8') as f:
    text_lenta = f.read()

with open(fname_2ch, 'r', encoding='utf-8') as f:
    text_2ch = f.read()

In [12]:
len(text_lenta), text_lenta[:100]

(11536552,
 'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к')

In [13]:
len(text_2ch), text_2ch[:100]

(11638405,
 ' Анимублядский WebM-треддля приличных анимублядей и прочих аутистов. Безграмотное быдло с дубляжом, ')

In [14]:
# Сокращаю размеры текстов, чтобы уместить в память
text_lenta = text_lenta[:100000]
text_2ch = text_2ch[:100000]

# Задание 1. (5 баллов) 
В тетрадке реализована биграмная языковая модель (при генерации учитывается информация только о 1 предыдущем слове). Реализуйте триграмную модель и сгенерируйте несколько текстов. Сравните их с текстами, сгенерированными биграмной моделью. 
Можно использовать те же тексты, что в семинаре, или взять какой-то другой (на английском или русском языке).

In [15]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

def ngrammer(tokens, n=3):
    """
    Get ngrams from list of tokens
    """
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

def create_ngrams(sentences):
    """
    Create unigrams, bigrams and trigrams from list of sentences
    """
    unigrams, bigrams, trigrams = Counter(), Counter(), Counter()
    for sentence in sentences:
        unigrams.update(sentence)
        bigrams.update(ngrammer(sentence, n=2))
        trigrams.update(ngrammer(sentence, n=3))
    return unigrams, bigrams, trigrams

In [16]:
sentences_dvach = [ ['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(text_2ch)]
sentences_news = [ ['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(text_lenta)]

unigrams_dvach, bigrams_dvach, trigrams_dvach = create_ngrams(sentences_dvach)
unigrams_news, bigrams_news, trigrams_news = create_ngrams(sentences_news)

In [17]:
def create_matrix(unigrams, bigrams, trigrams):
    """
    Based on unigrams, bigrams and trigrams this function creates matrix of trigrams, bigrams and supporting indexes
    """
    
    matrix_trigrams = np.zeros((len(bigrams), len(unigrams)))
    matrix_bigrams = np.zeros((len(unigrams), len(unigrams)))

    print("Initialized matrix")
    id2bigram = list(bigrams)
    bigram2id  =  {bigram:i for i, bigram in enumerate(id2bigram)}
    id2word = list(unigrams)
    word2id = {word:i for i, word in enumerate(id2word)}

    print('Start to fill the matrix_trigrams')
    for ngram in trigrams:
        ngram_ = ngram.split(' ')
        bigram, unigram = ' '.join(ngram_[:-1]), ngram_[-1]
        matrix_trigrams[bigram2id[bigram]][word2id[unigram]] =  (trigrams[ngram] / bigrams[bigram])
    print('Matrix is filled')

    print('Start to fill the matrix_bigrams')
    for ngram in bigrams:
        word1, word2 = ngram.split(' ')
        matrix_bigrams[word2id[word1]][word2id[word2]] =  (bigrams[ngram] / unigrams[word1])
    print('Matrix is filled')
    
    return matrix_trigrams, matrix_bigrams, id2bigram, bigram2id, id2word, word2id 

In [18]:
matrix_trigrams_dvach, matrix_bigrams_dvach, id2bigram_dvach, bigram2id_dvach, id2word_dvach, word2id_dvach = create_matrix(unigrams_dvach, bigrams_dvach, trigrams_dvach)

Initialized matrix
Start to fill the matrix_trigrams
Matrix is filled
Start to fill the matrix_bigrams
Matrix is filled


In [19]:
matrix_trigrams_news, matrix_bigrams_news, id2bigram_news, bigram2id_news, id2word_news, word2id_news = create_matrix(unigrams_news, bigrams_news, trigrams_news)

Initialized matrix
Start to fill the matrix_trigrams
Matrix is filled
Start to fill the matrix_bigrams
Matrix is filled


In [20]:
def generate_bigrams(matrix, id2word, word2id, n=100, start='<start>'):
    """
    Text generation based on bigrams matrix
    """
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id[start]
        current_idx = chosen
    
    return ' '.join(text)


def generate_trigrams(matrix, id2word, ngram2id, id2ngram, n=100, start='<start> <start>'):
    """
    Text generation based on trigrams matrix
    """
    text = []
    current_idx = ngram2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            token = start
        else:
            token = id2ngram[current_idx].split()[-1] + " " + id2word[chosen]
        current_idx = ngram2id[token]
    
    return ' '.join(text)

In [22]:
def compare(matrix_bigrams, matrix_trigrams, id2word, word2id, bigram2id, id2bigram, n=100):
    """
    Compare bigrams and trigrams texts
    """
    print('====================Bigram Texts ==============================')
    print(generate_bigrams(matrix=matrix_bigrams, id2word=id2word, word2id=word2id).replace('<end>', '\n'))
    print()
    
    print('====================Trigram Texts ==============================')
    print(generate_trigrams(matrix=matrix_trigrams, id2word=id2word, ngram2id=bigram2id, id2ngram=id2bigram).replace('<end>', '\n'))    
    print() 

In [23]:
# Сравниваем Двач
compare(matrix_bigrams_dvach, matrix_trigrams_dvach, id2word_dvach, word2id_dvach, bigram2id_dvach, id2bigram_dvach)

====================Bigram Texts ==============================
<start> <start> марксизм сталинизм 
 <start> <start> <start> <start> глянь автора artofeel 
 <start> с западными заказчиками со своим пофигизмом 
 ещё что-нибудь умное спиздани и настройке кодеков на танцы всем рекомендую 
 <start> табита кстати топ-20 отличная таблица выбора языка для работы кроме тебя подменю и стараются создать максимум 28 и бог с кем попиздеть 
 они разговариваат о чем mit дауны сидят и понял 
 <start> <start> <start> давай я понял 
 хоть историю гугла чтоле лол бля соус музыки 
 <start> <start> у вас вобьют хотя и все 28 какой-то 
 предлагаю скидывать сюда всякие цуиь и

====================Trigram Texts ==============================
а теперь последний вопрос 
 у них коэффициент интеллекта iq 180 знаете я расскажу одну историю 
 так как они юрисношаются 
 яндекс 41001634460485 похуй он пустой wmrr 944125761655 wm фейкижду с того что теперь бакалавриат а не писать об этом 
 я никогда особо не был таки

In [24]:
# Сравниваем Новости
compare(matrix_bigrams_news, matrix_trigrams_news, id2word_news, word2id_news, bigram2id_news, id2bigram_news)

====================Bigram Texts ==============================
<start> но с орбиты и автомобили 
 в операциях проводимых через лес до 140 тысячлет 
 королевская семья отметит траурную годовщину церемонией в 21 человека госпитализировано 
 <start> новый повод для того как утверждает intel их абазинский черкесский и удерживающими 12 заложников освобождены минувшей ночью на пресечение попыток привнесения элементов политического экстремизма а сразу на манежной площади не пострадали 
 <start> <start> <start> новое расследование 
 <start> все от залива ла-плата 
 сегодня была одобрена советом руководителей таможенных пунктах на повестке дня рождения м ю 
 <start> <start> <start> <start> <start> как указывается в убежище сектантов во всех избирателей 


====================Trigram Texts ==============================
однако число пострадавших в результате этого взрыва может составить до ста человек 
 при продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей захв

В обоих текстах очевидно, что с помощью триграммной матрицы генерируются более осмысленные тексты (как синтаксически, так и семантически). 

# Задание 2. (5 баллов) 
Напишите функцию оценивания нграммов на основе PMI. Используйте эту функцию вместо дефолтной в gensim.models.Phrases Обучите два последовательных модели Phrases с такой функцией и проанализируйте результаты, получаемые после преобразования текстов двумя Phrases.

Пояснения:
Формулу PMI можно посмотреть вот тут https://en.wikipedia.org/wiki/Pointwise_mutual_information , также там описано как вывести вероятности из количества вхождений слова1, слова2, нграмма и размера корпуса.
Чтобы функцию можно было поставить в аргумент scoring в Phrases у нее должны быть вот такие аргументы - scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count) 
Подберите параметр threshold под эту функцию. Чтобы проверить, что она вообще работает поставьте какое-то совсем маленькое число, чтобы порог проходили все нграммы и потом постепенно его повышайте.
В тетрадке есть пример обучения нескольких Phrases последовательно, воспользуйтесь им.

In [25]:
# https://github.com/RaRe-Technologies/gensim/blob/1d0a8bddc04090d07f55b8830ba451789b3fb7d0/gensim/models/phrases.py 
# Тут за основу взят normalized pmi из генсима

def pmi_score(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
    if bigram_count >= min_count:
        try:
            prob_a = worda_count / corpus_word_count
            prob_b = wordb_count / corpus_word_count
            prob_ab = bigram_count / corpus_word_count
            pmi = np.log(prob_ab / (prob_a * prob_b))
        except ZeroDivisionError:
            pmi = float('-inf') 
    else:
        pmi = float('-inf') 
    return pmi

In [26]:
pmi_score(10, 11, 9, 5, 0, 2000000), pmi_score(10, 11, 9, 5, 0, 200)

(12.005401950068022, 2.7950615780918397)

In [27]:
with open(fname_lenta, 'r', encoding='utf-8') as f:
    text_lenta = f.read()

with open(fname_2ch, 'r', encoding='utf-8') as f:
    text_2ch = f.read()
    
sentences_dvach = [normalize(text)for text in sent_tokenize(text_2ch)]
sentences_news = [normalize(text)for text in sent_tokenize(text_lenta)]

In [28]:
# default scoring
ph_default = gensim.models.Phrases(sentences_dvach, threshold=0.0001)
p_default = gensim.models.phrases.Phraser(ph_default)

# собираем статистики по уже забиграммленному тексту
ph2_default = gensim.models.Phrases(p_default[sentences_dvach])
p2_default = gensim.models.phrases.Phraser(ph2_default)
for i in  [50, 29, 23, 566, 491, 94, 222]:
    print(p2_default[p_default[sentences_dvach[i]]])

['вы', 'ничего_не', 'понимаете', 'в', 'настоящей', 'игре', 'что_значит', 'например_где']
['вчера', 'этой', 'не_было', 'пришлось', 'самому', 'резать']
['дропнул', 'из-за', 'ссыкухи']
['рады', 'чистить', 'ваши', 'конюшни', 'сэр']
['хардкор']
['так_что', 'я_за', 'ту', 'с', 'непроизносимым', 'именем', 'в_15', 'слов']
['рисовка', 'как_у', 'дитей', 'даунов_в', '5_лет', 'это', 'оригинал', 'манги', 'ванпанчмена']


In [29]:
# Наш PMI
ph_custom = gensim.models.Phrases(sentences_dvach, scoring=pmi_score, threshold=0.0001)
p_custom = gensim.models.phrases.Phraser(ph_custom)

# собираем статистики по уже забиграммленному тексту
ph2_custom = gensim.models.Phrases(p_custom[sentences_dvach], scoring=pmi_score)
p2_custom = gensim.models.phrases.Phraser(ph2_custom)
for i in  [50, 29, 23, 566, 491, 94, 222]:
    print(p2_custom[p_custom[sentences_dvach[i]]])

['вы_ничего', 'не_понимаете', 'в', 'настоящей', 'игре', 'что_значит', 'например_где']
['вчера', 'этой', 'не_было', 'пришлось', 'самому', 'резать']
['дропнул', 'из-за', 'ссыкухи']
['рады', 'чистить', 'ваши', 'конюшни', 'сэр']
['хардкор']
['так_что', 'я', 'за_ту', 'с', 'непроизносимым', 'именем', 'в_15', 'слов']
['рисовка', 'как_у', 'дитей', 'даунов_в', '5_лет', 'это', 'оригинал', 'манги', 'ванпанчмена']


In [33]:
ph_default = gensim.models.Phrases(sentences_dvach, threshold=13)
p_default = gensim.models.phrases.Phraser(ph_default)

# собираем статистики по уже забиграммленному тексту
ph2_default = gensim.models.Phrases(p_default[sentences_dvach])
p2_default = gensim.models.phrases.Phraser(ph2_default)
for i in  [50, 29, 23, 566, 491, 94, 222]:
    print(p2_default[p_default[sentences_dvach[i]]])

['вы', 'ничего_не', 'понимаете', 'в', 'настоящей', 'игре', 'что', 'значит', 'например', 'где']
['вчера', 'этой', 'не', 'было', 'пришлось', 'самому', 'резать']
['дропнул', 'из-за', 'ссыкухи']
['рады', 'чистить', 'ваши', 'конюшни', 'сэр']
['хардкор']
['так', 'что', 'я', 'за', 'ту', 'с', 'непроизносимым', 'именем', 'в', '15', 'слов']
['рисовка', 'как', 'у', 'дитей', 'даунов', 'в', '5_лет', 'это', 'оригинал', 'манги', 'ванпанчмена']


In [34]:
# Наш PMI
ph_custom = gensim.models.Phrases(sentences_dvach, scoring=pmi_score, threshold=13)
p_custom = gensim.models.phrases.Phraser(ph_custom)

# собираем статистики по уже забиграммленному тексту
ph2_custom = gensim.models.Phrases(p_custom[sentences_dvach], scoring=pmi_score)
p2_custom = gensim.models.phrases.Phraser(ph2_custom)
for i in  [50, 29, 23, 566, 491, 94, 222]:
    print(p2_custom[p_custom[sentences_dvach[i]]])

['вы', 'ничего', 'не', 'понимаете', 'в', 'настоящей', 'игре', 'что', 'значит', 'например', 'где']
['вчера', 'этой', 'не', 'было', 'пришлось', 'самому', 'резать']
['дропнул', 'из-за', 'ссыкухи']
['рады', 'чистить', 'ваши', 'конюшни', 'сэр']
['хардкор']
['так', 'что', 'я', 'за', 'ту', 'с', 'непроизносимым', 'именем', 'в', '15', 'слов']
['рисовка', 'как', 'у', 'дитей', 'даунов', 'в', '5', 'лет', 'это', 'оригинал', 'манги', 'ванпанчмена']


В данном пункте я взяла только тексты двача. Можем также заметить, что дефолтный scoring показывает лучше результаты, чем наш pmi с обеими версиями threshold'а. Наш pmi не выделает нграмы если мы повышаем порог.